<center>
<p><img src="https://mcd.unison.mx/wp-content/themes/awaken/img/logo_mcd.png" width="150">
</p>



<h1>Curso Ingeniería de Características</h1>

<h3>Datos cualitativos y cuantitativos</h3>


<p> Julio Waissman Vilanova </p>
<p>
<img src="https://identidadbuho.unison.mx/wp-content/uploads/2019/06/letragrama-cmyk-72.jpg" width="150">
</p>


<a target="_blank" href="https://colab.research.google.com/github/mcd-unison/ing-caract/blob/main/ejemplos/tipos/python/catnum.ipynb"><img src="https://i.ibb.co/2P3SLwK/colab.png"  style="padding-bottom:5px;"  width="30" /> Ejecuta en Colab</a>

</center>

Esta libreta es una modificación del tutorial de `sklearn`:

[**Comparing Target Encoder with Other Encoders**](https://scikit-learn.org/stable/auto_examples/preprocessing/plot_target_encoder.html#sphx-glr-auto-examples-preprocessing-plot-target-encoder-py)


### 1. Cargando datos de OpenML

Vamos a bajar datos del repositorio de datos `OpenML` (algo así como bajarlos de *Kaggle*) para buscar uno de los conjuntos de datos mas sobados en Ciencia de Datos. 

Así es, como imaginas, estamos hablando del conjunto de evaluaciones de Vinos.

In [ ]:
from sklearn.datasets import fetch_openml

wine_reviews = fetch_openml(data_id=42074, as_frame=True, parser="pandas")

df = wine_reviews.frame
df.head()

### 2. Separando las características por tipo

Vamos a separar la variable objetivo, las numéricas y las categóricas

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

pd.options.display.float_format = '{:.2f}'.format # Se lee mejor
plt.style.use('ggplot')

numericas_c = ["price"]
categoricas_c = [
    "country",
    "province",
    "region_1",
    "region_2",
    "variety",
    "winery",
]
target_c = "points"

X = df[numericas_c + categoricas_c]
y = df[target_c]

y.plot(
    kind='hist',
    title='Distribución de la salida',
    xlabel='points',
    ylabel='accumulados'
)

X.head()

### 3. Codificación sencilla de variables categoricas

En esta celda vamos a codificar las variables categoricas solamente, y ver como se pueden juntar diferentes codificadores para hacer un solo objeto, el cual se podrá integrar en todos los modelos de `sklearn` usando un `pipeline`. 

Vamos a empezar definiendo un codificador muy sencillo.

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder

ordinal_encoder = OrdinalEncoder(
    handle_unknown="use_encoded_value", 
    unknown_value=-1
)

preprocessor = ColumnTransformer(
    [
        ("numericas", "passthrough", numericas_c),
        ("categorical", ordinal_encoder, categoricas_c),
    ]
)

preprocessor

In [ ]:
X_codificada = preprocessor.fit_transform(X)

X_cod = pd.DataFrame(
    X_codificada, 
    columns=preprocessor.get_feature_names_out()
)
X_cod.head()


In [ ]:
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler

minmax_scaler = MinMaxScaler(
    feature_range=(-1, 1)
)

onehot_encoder = OneHotEncoder(
    handle_unknown="ignore", 
    max_categories=20, 
    sparse_output=False
)

preprocessor2 = ColumnTransformer(
    [
        ("numericas", minmax_scaler, numericas_c),
        ("categorical", onehot_encoder, categoricas_c),
    ]
)

preprocessor2

In [ ]:
X_codificada = preprocessor2.fit_transform(X)

X_cod = pd.DataFrame(
    X_codificada, 
    columns=preprocessor2.get_feature_names_out()
)
X_cod

### 4. Codificación de variables categóricas mas elaborada

El siguiente diagrama da una buena idea de como tratar a los datos categóricos, ya una ves que decidamos que vamos a realizar un método de aprendizaje.

![](./cat-flowchart.png)

Y por lo que dice aqui, pues es importante saber cuales atributos tienen muchas categorias

In [ ]:
n_unique_categories = (
    df[categoricas_c]
    .nunique()
    .sort_values(ascending=False)
)
display("Numero de categorias diferentes por característica")
display(n_unique_categories)

Por lo que vamos a separar las variables en `alta_cardinalidad`, `media_cardinalidad` y `baja_cardinalidad`

In [ ]:
alta_cardinalidad = n_unique_categories[n_unique_categories > 255].index
media_cardinalidad = n_unique_categories[
    n_unique_categories[20 < n_unique_categories] & 
    n_unique_categories[n_unique_categories <= 255]
].index
baja_cardinalidad = n_unique_categories[n_unique_categories <= 20].index


print(f'Variables de alta cardinalidad: {alta_cardinalidad.values}')
print(f'Variables de media cardinalidad: {media_cardinalidad.values}')
print(f'Variables de baja cardinalidad: {baja_cardinalidad.values}')

Ahora si vamos a hacer algo mas macabron:

In [ ]:
from sklearn.preprocessing import PowerTransformer, TargetEncoder

normalizador = PowerTransformer(method="yeo-johnson")

onehot_encoder_2 = OneHotEncoder(
    handle_unknown="ignore", 
    sparse_output=False
)
target_encoder = TargetEncoder(target_type="continuous")


preprocessor3 = ColumnTransformer(
    [
        ("numerical", normalizador, numericas_c),
        ("alta_cardinalidad", target_encoder, alta_cardinalidad),
        ("media_cardinalidad", ordinal_encoder, media_cardinalidad),
        ("baja_cardinalidad", onehot_encoder_2, baja_cardinalidad),
    ],
)
preprocessor3

In [ ]:

X_codificada = preprocessor3.fit_transform(X, y)

X_cod = pd.DataFrame(
    X_codificada, 
    columns=preprocessor3.get_feature_names_out()
)
X_cod

### 5. Haciendo un clasificador con esto

Ahora vamos a usar todo junto, la codificación con un modelo de aprendizaje, tal como se usa en `sklearn`

In [ ]:
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.pipeline import make_pipeline

regresor = HistGradientBoostingRegressor(
    random_state=0, 
    max_iter=20, 
)

modelo1 = make_pipeline(
    preprocessor,
    regresor,
)
modelo1

In [ ]:
modelo2 = make_pipeline(
    preprocessor2,
    regresor,
)
modelo2

In [ ]:
modelo3 = make_pipeline(
    preprocessor2,
    regresor,
)
modelo3

In [ ]:
modelo1.fit(X, y)
modelo2.fit(X, y)
modelo3.fit(X, y)

In [ ]:
scores = [
    modelo1.score(X, y),
    modelo2.score(X, y),
    modelo3.score(X, y)
]
scores